## Packages Requirements

In [ ]:
%%capture --no-stderr
%pip install scrapegraphai langchain_scrapegraph dotenv pandas qdrant-client fastembed nest_asyncio
#playwright install
%pip install
%pip install -U duckduckgo-search
%pip install scrapegraphai'[other-language-models]'
%pip install scrapegraphai'[more-semantic-options]'
%pip install scrapegraphai'[more-browser-options]'

## 1. SmartScraperGraph with OpenAI

In [ ]:
import os
from dotenv import load_dotenv

import json
from scrapegraphai.graphs import SmartScraperGraph, DepthSearchGraph
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# Define the configuration for the scraping pipeline
graph_config = {
    "llm": {
        "api_key": api_key,
        "model": "openai/gpt-4o-mini",
    },
    "verbose": True,
    "headless": True,
    "depth": 2,
    "only_inside_links": False
}

# Create the SmartScraperGraph instance
smart_scraper_graph = SmartScraperGraph(
    prompt="Extract all the posts from the website",
    source="https://www.aivi.fyi/",
    config=graph_config
)
# Run the pipeline
result = smart_scraper_graph.run()
print(json.dumps(result, indent=4,ensure_ascii=False))


# search_graph = DepthSearchGraph(
#     prompt="List me all the projects with their description",
#     source="https://perinim.github.io",
#     config=graph_config
# )
# result = search_graph.run()
# print(result)

一名打算獨自出國旅行的上班族，考慮到安排行程都需要花很多時間搜集資料，打算參考日本在地的旅遊網站內容(https://osaka.letsgojp.com/archives/548388/)，來安排京都熱門的旅遊景點。使用python開發，並透過BeautifulSoup套件來擷取，該網頁內容景點相關的內容。 每次都會隨機提供三個熱門景點與相關參考資訊，"如果不在推薦景點內，系統會另外提供其他熱門景點。只有顯示網頁回應200時才會開始推薦。每次執行都重新讀取網頁資訊，不需要存檔。

1.一名計劃獨自出國旅行的上班族，考慮到安排行程需要花費大量時間搜集資料，決定參考日本在地的旅遊網站內容。
2.使用Python開發，並透過BeautifulSoup套件來擷取該網頁中與景點相關的內容。
3.每次都會隨機提供三個熱門景點與相關參考資訊。如果不在推薦景點內，系統會另外提供其他熱門景點。
4.只有在網頁回應狀態碼為200時才會開始推薦。
5.每次執行時都會重新讀取網頁資訊，不需要存檔。

In [ ]:
import requests
from bs4 import BeautifulSoup
import random

# 參考網址
url = 'https://osaka.letsgojp.com/archives/548388/'

# 發送 GET 請求
response = requests.get(url)
response.encoding = 'utf-8'

# 確認請求成功
if response.status_code == 200:
    # 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 擷取文章主體
    article = soup.find('div', class_='entry-content clearfix')

    # 初始化資料列表
    playinfo=[]
    data=[]
    planinfo=[]
    # 提取標題
    title = soup.find('h1', class_='h3 my-3').text.strip()


    # 提取景點資訊
    location_data = article.find_all('p', {'align': 'justify'})

    for locations in location_data:
        # 取得景點名稱
        span_tag = locations.find('span')
        names = span_tag.text.strip() if span_tag else None

        # 取得景點資訊
        info = locations.get_text(strip=True)
        if names:
            info = info.replace(names, "")  # 移除名稱
        info = info.replace('"', '').replace("\r\n", "").replace("\r", "").replace("\n", "").replace("・", " ❊ ")

        # 取得官方網站連結
        a_tag = locations.find('a')
        web_url = a_tag['href'] if a_tag else None
        if web_url:
            info = info.replace("官方網站", f"官方網站：{web_url}")

        # 存入 `playinfo`
        if names:
            playinfo.append([names, info])

        #隨機推薦三個景點
        if len(playinfo) >= 3:
            visit_places = random.sample(playinfo, 3)
        else:
            visit_places = playinfo  # 如果不足 3 個，就顯示所有

    # 顯示隨機推薦的 3 個景點
    print(f"根據'{title}'網站介紹，隨機推薦的 3 個景點：\n")
    for place in visit_places:
        data.append(place[0])
        planinfo=f"景點：{place[0]}\n參考資訊:{place[1]}\n"
        planinfo=planinfo.replace(" ❊ ", " \n ❊ ")
        print(f'{planinfo}')


    # 提取 playinfo 裡的景點名稱
    planinfo = {item[0] for item in playinfo}

    # 找出不在推薦景點中的名稱
    extra_places = [name for name in planinfo if name not in data]
    print("\n下次旅行可參考的地點:\n", "、".join(extra_places))

else:
    print(f'無法取得網頁內容，狀態碼：{response.status_code}')